In [ ]:
%pip install -q transformers

In [ ]:
from dalle_tiny.model import TinyDalleModel
from dalle_tiny.util import TinyDalleDataset
import torch
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.cuda.manual_seed_all(42)
torch.manual_seed(42)

training_data = TinyDalleDataset(parquet_file="https://github.com/cthiounn/dalle-tiny/raw/main/archive_train.parquet",dataset_type="train")
#test_data = TinyDalleDataset(csv_file="validation.csv",dataset_type="val")

train_dataloader = DataLoader(training_data, batch_size=10, shuffle=True)


In [ ]:
from tqdm import tqdm
from transformers import BartForConditionalGeneration
import gc

def freeze_params(model):
    for par in model.parameters():
        par.requires_grad = False

#del model
gc.collect()
torch.cuda.empty_cache()
model=TinyDalleModel.from_pretrained('facebook/bart-large-cnn')
model.reinit_lm_head_for_images()
model=model.to(device)
freeze_params(model.get_encoder())
model.train()

for epoch in range(5):
    total_loss=0
    for batch in tqdm(train_dataloader):
        caption,label =batch
        inp=caption.to(device)
        lab=label[0].to(device)
        predict=model(input_ids=inp, decoder_input_ids =lab)
        print(predict)
        break